## Blending for Rekko

Чтобы повысить скор, надо объединять результаты предсказаний от разных моделей

### 1 способ "shuffle"

Этот способ заключается в следующем. Берется два сабмита. Один сабмит хорошим скором от одной модели, и второй сабмит с чуть меньшим скором, но от другой модели. Далее для каждого пользователя находится пересечение фильмов. Те фильмы, в первом сабмите, которые есть во втором, остаются на своих местах. А вот фильмы, которые присутствуют только в первом, "вылетают" со своих мест и занимают оставшиеся

In [33]:
import numpy as np
import json
d1 = {}
d2 = {}
with open('./share/2_5312170359494017878.json', 'r') as f:
    d2 = json.load(f)
    
with open('./share/nn_light_bm.json', 'r') as f:
    d1 = json.load(f)

In [34]:
len(d2)

50000

In [35]:
av = []
d = {}
counter = 0

for key in d1.keys():
    if key in d2.keys():
        ar1 = np.array(d1[key])
        ar2 = np.array(d2[key])
        inter = np.intersect1d(ar1,ar2)
        av.append(len(inter))
        to_add = []
        for el in ar2:
            if el  in inter:
                to_add.append(el)
        for el in ar2:
            if el  not in to_add:
                to_add.append(el)
        
        
        to_add = np.array(to_add)
        
        d[key] = list(map(int,list(to_add)))

    else:
        d[key] = d1[key]

  

with open('1.json', 'w') as f:
    json.dump(d, f)
    
print('average intersaction equals {},{}'.format(np.array(av).mean(),np.array(av).var()))

average intersaction equals 9.309129171259645,7.664415104970264


In [9]:
len(d)

49895

### Второй метод

Заключается в суммировании общего рейтинга и дальнейший сортировки по нему

In [77]:
with open('./share/bm25_last_submit.json', 'r') as f:
    d2 = json.load(f)
    
with open('./share/blend.json', 'r') as f:
    d1 = json.load(f)
    
with open('./share/answer_50.json', 'r') as f:
    d4 = json.load(f)
    
with open('./share/rf.json', 'r') as f:
    d5 = json.load(f)
    
with open('./share/2_5312170359494017878.json', 'r') as f:
    d3 = json.load(f)

In [78]:
sol_merged = dict()
lens = []
for user, els1 in d1.items():
    merget_score = defaultdict(int)

    els2 = d2[user]
    els3 = d3[user]
    els4 = d4[user]
    els5 = d5[user]
    union_el = set(els1).union(els2).union(els3).union(els4).union(els5)
    for i, el2 in enumerate(els2):
        merget_score[el2] += i- 1
    for el in union_el.difference(els2):
        merget_score[el] += 19
        
    for i, el3 in enumerate(els3):
        merget_score[el3] += i
    for el in union_el.difference(els3):
        merget_score[el] += 20
        
    for i, el4 in enumerate(els4):
        merget_score[el4] += i+1
    for el in union_el.difference(els4):
        merget_score[el] += 21
        
    for i, el5 in enumerate(els5):
        merget_score[el5] += i+2
    for el in union_el.difference(els5):
        merget_score[el] += 22
        
        
    current_top = list(zip(*sorted(merget_score.items(),key=lambda x:x[1])))[0]
    lens.append(len(current_top))
    sol_merged[user] = current_top[:20]

In [79]:
with open('./share/final.json', 'w') as f:
    json.dump(sol_merged, f)

In [2]:
# считаем map относительно лучшего сабмита
def validation(dict_):
    d1 = {}
    with open('./share/2_5312170359494017878.json', 'r') as f:
        d1 = json.load(f)
    av = []


    for key in d1.keys():
        if key in dict_.keys():
            ar1 = np.array(d1[str(key)])
         # print(ar1)
            ar2 = np.array(dict_[str(key)])
            inter = np.intersect1d(ar1,ar2)
        #  av.append(len(inter))
            av.append(mapk([ar1],[ar2]))
            to_add = []
            for el in ar1:
                if el  in inter:
                    to_add.append(el)
            for el in ar1:
                if el  not in to_add:
                    to_add.append(el)


            to_add = np.array(to_add)
    return np.mean(av)



def mapk(actual, predicted, k=20):
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])
  
def apk(actual, predicted, k=10):

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

In [80]:
validation(sol_merged)

0.5814547965520875